# Scrapy test for pubMed 

1. Scrape the page for links with articles from https://www.ncbi.nlm.nih.gov/pmc/
2. Scrape each article for histological images and their caption
3. Save the found data into single html file;

In [1]:
### Import Scrapy
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.4'

In [2]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
import logging

from twisted.internet import reactor, defer
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging

In [3]:
### global variables
root_url = 'https://www.ncbi.nlm.nih.gov'
articleUrls = []
articleUrlsCount = 0
imagesCount = 0 


In [4]:
#defining the spider for collecting articles

class pubMedArticlesSpider(scrapy.Spider):
    name = "pubMedArticles"
    start_urls = [
        'https://ncbi.nlm.nih.gov/pmc/issues/359201/'
    ]
    
    def parse(self, response):
        urls = response.xpath('//div[contains(@class, "title")]/a/@href')        
        n = len(urls)
        #print("{articleUrlsCount} articles are found".format(articleUrlsCount=n))        
        for article in urls:
            url =  root_url + article.get()
            articleUrls.append(url)

        articleUrlsCount = len(articleUrls)
        print("articlesCount: ", articleUrlsCount)


In [5]:

#defining the public medical images spider
class medImageSpider(scrapy.Spider):
    name = "medPubImages"
    start_urls = articleUrls
#     [
#         #'https://pubmed.ncbi.nlm.nih.gov/',
#         'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4924648/',
#         'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6024750/'
#         #'https://www.ncbi.nlm.nih.gov/pmc/?term=(cancer)+AND+lung'
#     ]
#    print(start_urls)
    def parse(self, response):
        #html = "<!DOCTYPE html><html><head></head><body>"
        html =""
        articleUrl = response.request.url
        print ("URL: " + articleUrl)
        docTitle = response.xpath('//h1[contains(@class, "content-title")]').get()
        print("docTitle: " + docTitle)

        docId = articleUrl.split("/")
        #print(docId)
        docId = docId[len(docId)-2]
        print("docId: " + docId)
        figures = response.xpath('//div[contains(@class, "fig ")]')
        
        n = len(figures)
        print("{n} figures are found".format(n=n))
        i = 0 
        for figure in figures:
            img_link = root_url + figure.xpath('.//img/@src').get()
            #print(img_link)
            text = figure.xpath('.//div[contains(@class, "caption")]').get()
            #print(text)
            html += """{i}<div style="font-size: 50%;" ><a href="{article_url}" target="_blank">Article ID: {docId} {title}</a><div class="fig"><a href="{url}" target="_blank"> 
                <img src="{url}" style="width: 45%; height: 45%;" />
                <a/>{text}</div></div><hr>""".format(i=i,article_url=articleUrl,docId=docId, title=docTitle,url=img_link, text=text)  
            i += 1
        #html += "</body></html>"
        with open("medImages4.html", "a", encoding='utf-8') as page:
            page.write(html)
            page.close()
            

In [6]:
### Start the crawlers
configure_logging()
runner = CrawlerRunner({
                'USER_AGENT': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'
            })

@defer.inlineCallbacks
def crawl():
    yield runner.crawl(pubMedArticlesSpider)
    print("{articleUrlsCount} articles are found".format(articleUrlsCount = len(articleUrls)))
    print(articleUrls)
    yield runner.crawl(medImageSpider)
    reactor.stop()

crawl()
reactor.run()


2021-01-17 16:11:41 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (compatible; Googlebot/2.1; '
               '+http://www.google.com/bot.html)'}
2021-01-17 16:11:41 [scrapy.extensions.telnet] INFO: Telnet Password: 1632ea5b749f46d2
2021-01-17 16:11:41 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-01-17 16:11:42 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scr

2021-01-17 16:11:42 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.ncbi.nlm.nih.gov/pmc/issues/359201/> from <GET https://ncbi.nlm.nih.gov/pmc/issues/359201/>
2021-01-17 16:11:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/issues/359201/> (referer: None)
2021-01-17 16:11:43 [scrapy.core.engine] INFO: Closing spider (finished)
2021-01-17 16:11:43 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 548,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 8386,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/301': 1,
 'elapsed_time_seconds': 1.64882,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 1, 17, 14, 11, 43, 833271),
 'log_count/DEBUG': 2,
 'log_count/INFO': 10,
 'response_received_count': 1,
 'scheduler/dequeued': 2,
 'scheduler/dequeued/mem

articlesCount:  11
11 articles are found
['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279775/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279772/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279768/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279771/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279774/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279773/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279769/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279778/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279777/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279776/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279770/']


2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279775/> (referer: None)
2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279774/> (referer: None)
2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279768/> (referer: None)
2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279771/> (referer: None)


URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279775/
docTitle: <h1 class="content-title">NOVEL <em>XRCC4</em> MUTATIONS IN AN INFANT WITH MICROCEPHALIC PRIMORDIAL DWARFISM, DILATED CARDIOMYOPATHY, SUBCLINICAL HYPOTHYROIDISM, AND EARLY DEATH: EXPANDING THE PHENOTYPE OF <em>XRCC4</em> MUTATIONS</h1>
docId: PMC7279775
1 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279774/
docTitle: <h1 class="content-title">HYPOTHYROIDISM AND GOITER IN A YOUNG MALE WITH SUSPECTED DIETARY IODINE DEFICIENCY FOLLOWED BY THYROTOXICOSIS AFTER IODINE SUPPLEMENTATION</h1>
docId: PMC7279774


2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279769/> (referer: None)
2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279773/> (referer: None)
2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279772/> (referer: None)
2021-01-17 16:11:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279778/> (referer: None)


2 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279768/
docTitle: <h1 class="content-title">RECURRENT PAINFUL HASHIMOTO THYROIDITIS SUCCESSFULLY TREATED BY THYROIDECTOMY</h1>
docId: PMC7279768
2 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279771/
docTitle: <h1 class="content-title">THERAPEUTIC PLASMA EXCHANGE AS A BRIDGE TO TOTAL THYROIDECTOMY IN PATIENTS WITH SEVERE THYROTOXICOSIS</h1>
docId: PMC7279771
1 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279769/
docTitle: <h1 class="content-title">A UNIQUE CASE OF ATEZOLIZUMAB-INDUCED AUTOIMMUNE DIABETES</h1>
docId: PMC7279769
0 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279773/
docTitle: <h1 class="content-title">SPONTANEOUS RESOLUTION OF PRIMARY HYPERCORTISOLISM OF CUSHING DISEASE AFTER PITUITARY HEMORRHAGE</h1>
docId: PMC7279773
2 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279772/
docTitle: <h1 class="content-title

2021-01-17 16:11:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279776/> (referer: None)
2021-01-17 16:11:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279770/> (referer: None)
2021-01-17 16:11:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279777/> (referer: None)
2021-01-17 16:11:46 [scrapy.core.engine] INFO: Closing spider (finished)
2021-01-17 16:11:46 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 3102,
 'downloader/request_count': 11,
 'downloader/request_method_count/GET': 11,
 'downloader/response_bytes': 213562,
 'downloader/response_count': 11,
 'downloader/response_status_count/200': 11,
 'elapsed_time_seconds': 2.515345,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 1, 17, 14, 11, 46, 499008),
 'log_count/DEBUG': 11,
 'log_count/INFO': 10,
 'response_received_count': 11,

URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279776/
docTitle: <h1 class="content-title">A CASE OF ACQUIRED GENERALIZED LIPODYSTROPHY ASSOCIATED WITH PEMBROLIZUMAB IN A PATIENT WITH METASTATIC MALIGNANT MELANOMA</h1>
docId: PMC7279776
3 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279770/
docTitle: <h1 class="content-title">MANIFESTATIONS OF GLUCAGONOMA SYNDROME</h1>
docId: PMC7279770
4 figures are found
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7279777/
docTitle: <h1 class="content-title">LINAGLIPTIN-INDUCED PANCREATITIS</h1>
docId: PMC7279777
0 figures are found
